# Transformers pour la classification de texte

L'objectif de ce TP est d'implémenter une version simplifiée d'un Transformer pour résoudre un problème de classification de texte.

Nous utiliserons comme exemple illustratif une base de données présente dans la librairie ```Keras``` consistant en des critiques de films postées sur le site IMDB, accompagnées d'une note qui a été binarisée pour révéler le caracète positif, ou négatif, de la critique.

In [ ]:
+

2024-10-21 14:48:09.115655: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-21 14:48:09.125573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 14:48:09.137303: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 14:48:09.140692: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 14:48:09.149271: I tensorflow/core/platform/cpu_feature_guar

## Implémentation d'un bloc de base de Transformer


<center><img src="https://drive.google.com/uc?id=1leAVyTZJ2gZ26CFoauMtmu7T4Jo-Crc_" width=200> </center>
<caption><center> Figure 1: Schéma de l'architecture de BERT</center></caption>

La figure ci-dessus présente l'architecture de BERT. Le bloc de base d'un Transformer est composé d'un bloc de *Self-Attention*, d'une couche de ```Layer Normalization``` (similaire à la ```Batch Normalization```), d'une couche dense et enfin d'une nouvelle couche de ```Layer Normalization```.

Pour implémenter la *Self-Attention*, vous pouvez utiliser la fonction ```Multi-Head Attention``` (à vous de regarder quels en sont les paramètres dans la documentation).

**Rappel**: Une couche d'Attention *Multi-Head*  se présente sous la forme ci-dessous à gauche, avec le mécanisme d'attention détaillé à droite :


<center>

<img src="https://drive.google.com/uc?id=1UTozEHtsZ3xy61XJqn_Eug-7mn7bFp9m">
<img src="https://drive.google.com/uc?id=1aTttpp1OOasVVZAi3lWwosh68VnBjQnz">
</center>

D'après vous, combien de paramètres comporte une couche d'attention à 2 têtes, pour un *Embedding* de dimension 32 ?

-> 2 head self attention -> pour chaques tokens, on veux 2 x 32 x 3 = 192 paramètres

BON BAH C'EST FAUX LOL
enfaite c'est pas 32 le nombre de param à multiplier parce que ya AUSSI la couche dense finale qui change de 64 à 32 paramètre, ET ya aussi la couche dense en préambule
on se retrouve donc avec
((32*32+32(biais)) x 3(QKV) x 2(têtes))+ (32 x 64 + 32 (biais)) = 8416 paramètres

In [2]:
class TransformerBlock(layers.Layer):
    # embed_dim désigne la dimension des embeddings maintenus à travers les différentes couches,
    # et num_heads le nombre de têtes de la couche d'attention.
    # DANS CETTE FONCTION, ON NE FAIT QUE DEFINIR LES COUCHES
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        # Définition des différentes couches qui composent le bloc
        # Couche d'attention
        self.att = layers.MultiHeadAttention(num_heads, embed_dim,value_dim=embed_dim)
        # Première couche de Layer Normalization
        self.layernorm1 = layers.LayerNormalization()
        # Couche Dense (Feed-Forward)
        self.ffn = layers.Dense(embed_dim, activation="softmax")
        # Deuxième couche de normalisation
        self.layernorm2 =layers.LayerNormalization()

    # DANS CETTE FONCTION, ON APPELLE EXPLICITEMENT LES COUCHES DEFINIES DANS __init__
    # ON PROPAGE DONC LES ENTREES inputs A TRAVERS LES DIFFERENTES COUCHES POUR OBTENIR
    # LA SORTIE
    def call(self, inputs):
        # Application des couches successives aux entrées
        x = self.att(inputs,inputs,use_causal_mask=True)
        y = self.layernorm1(x+ inputs)
        z = self.ffn(y)
        x = self.layernorm2(z+y)
        return x


## Implémentation de la double couche d'Embedding

La séquence d'entrée est convertie en *Embedding* de dimension ```embed_dim```.
L'*Embedding* final est constitué de la somme de deux *Embedding*, le premier encodant un mot, et le second encodant la position du mot dans la séquence.

La couche d'*Embedding* de Keras (```layers.Embedding```) est une sorte de table associant à un indice en entrée un vecteur de dimension ```embed_dim```. Chaque coefficient de cette table est en fait un paramètre apprenable.

D'après vous combien de paramètres contiendrait une couche d'*Embedding* associant un vecteur de dimension 32 à chacun des 20000 mots les plus courants du vocabulaire extrait de la base de données que nous allons utiliser ?
Et combien pour l'*Embedding* qui associe un vecteur de dimension 32 à chaque position d'un séquence de longueur ```maxlen``` ?

In [3]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        # Définition des différentes couches qui composent le bloc Embedding
        # Embedding de mot
        self.token_emb = layers.Embedding(vocab_size,embed_dim)
        # Embedding de position
        self.pos_emb = layers.Embedding(maxlen, embed_dim)

    def call(self, x):
        # Calcul de l'embedding à partir de l'entrée x
        # ATTENTION : UTILISER UNIQUEMENT DES FONCTIONS TF POUR CETTE PARTIE
        # Récupération de la longueur de la séquence
        # on a un vecteur de taille (1,maxlen)
        maxlen = tf.shape(x)[-1]
        # Création d'un vecteur [0, 1, ..., maxlen] des positions associées aux
        # mots de la séquence (fonction tf.range)
        positions = tf.range( maxlen )
        # Calcul des embeddings de position
        positions_emb = self.pos_emb(positions)
        # Calcul des embeddings de mot
        words_emb = self.token_emb(x)
        return positions_emb + words_emb

## Préparation de la base de données

In [4]:
# Taille du vocabulaire considéré (on ne conserve que les 20000 mots les plus courants)
vocab_size = 20000
# Taille maximale de la séquence considérée (on ne conserve que les 200 premiers mots de chaque commentaire)
maxlen = 200

# Chargement des données de la base IMDB
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)

print(len(x_train), "séquences d'apprentissage")
print(len(x_val), "séquences de validation")

# Padding des séquences : ajout de "0" pour compléter les séquences trop courtes
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
25000 séquences d'apprentissage
25000 séquences de validation


In [5]:
max(y_val)

1

## Création du modèle

Pour assembler le modèle final, il faut, partant d'une séquence de longueur ```maxlen```, calculer les Embedding puis les fournir en entrée d'une série de blocs Transformer. Pour ce TP, **commencez par ne mettre qu'un seul bloc Transformer**. Vous pourrez en ajouter plus tard si vous le souhaitez.

Pour construire la tête de projection du réseau, vous pouvez moyenner les activations en sortie du bloc Transformer par élément de la séquence grâce à un *Global Average Pooling* (1D !), à relier à une couche dense (par exemple comportant 20 neurones) et enfin à la couche de sortie du réseau.

In [6]:
embed_dim = 32  # Dimension de l'embedding pour chaque mot
num_heads = 2  # Nombre de têtes d'attention

# A COMPLETER
inputs = layers.Input(shape=(maxlen,))
embeddings = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)(inputs)
att = TransformerBlock(embed_dim, num_heads)(embeddings)
proj = layers.GlobalAveragePooling1D()(att)
post_processing = layers.Dense(20,activation="relu")(proj)
outputs = layers.Dense(1,activation="sigmoid") (post_processing)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

I0000 00:00:1729514894.080686   34125 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729514894.185234   34125 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729514894.186932   34125 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729514894.190346   34125 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, 200, 32)        │       646,400 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ (None, 200, 32)        │         9,600 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │           660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 656,681 (2.51 MB)

 Trainable params: 656,681 (2.51 MB)

 Non-trainable params: 0 (0.00 B)

Enfin vous pouvez lancer l'apprentissage, avec par exemple l'optimiseur Adam. Inutile de lancer de trop nombreuses *epochs*, le réseau sur-apprend très vite !

In [7]:
# A COMPLETER
model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val)
)

Epoch 1/5


I0000 00:00:1729514896.420606   34255 service.cc:146] XLA service 0x7f94f00136d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729514896.420627   34255 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-10-21 14:48:16.474008: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-21 14:48:16.675156: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-10-21 14:48:17.207113: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_90', 40 bytes spill stores, 40 bytes spill loads

2024-10-21 14:48:17.682774: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory 

 91/782 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5521 - loss: 0.6926

I0000 00:00:1729514899.813513   34255 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.7143 - loss: 0.5368 - val_accuracy: 0.8589 - val_loss: 0.3280
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9199 - loss: 0.2119 - val_accuracy: 0.8561 - val_loss: 0.3493
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9607 - loss: 0.1151 - val_accuracy: 0.8477 - val_loss: 0.4073
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9810 - loss: 0.0619 - val_accuracy: 0.8398 - val_loss: 0.5183
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9906 - loss: 0.0316 - val_accuracy: 0.8270 - val_loss: 0.6896


**Questions subsidiaires**:



1.   Testez un LSTM bi-directionnel, comme nous l'avons vu dans le TP précédent, et comparez les résultats obtenus sur ce problème.
2.   Reprenez le code du TP précédent et appliquez le modèle Transformer de type BERT au problème de classification de nom de famille.
3.   Faites ensuite la même chose pour le problème de génération de nom de famille.  
**ATTENTION : un modèle de type BERT n'est pas adapté à un problème de génération de texte. Pour cela il faut passer sur un modèle de type GPT qui utilise une couche d'auto-attention masquée, i.e. qui empêche un token de porter attention sur la suite de la séquence. Lisez bien la documentation de la couche d'Attention pour trouver comment faire cette modification.**

